In [23]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [33]:
%cd "/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier"

/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [44]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

# Loading Raw Data

In [45]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

In [46]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [47]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [48]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [49]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [50]:
X_train = np.concatenate((x_train_list[0][:200, :], x_train_list[1][:200, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[200:] += 1

X_train.shape, Y_train.shape

((400, 729), (400,))

In [51]:
X_test = np.concatenate((x_test_list[0][:500, :], x_test_list[1][:500, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[500:] += 1

X_test.shape, Y_test.shape

((1000, 729), (1000,))

# Dataset Preprocessing

In [52]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

# Quantum

In [53]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

qml.enable_tape()

from tensorflow.keras.utils import to_categorical

# Set a random seed
np.random.seed(2020)

In [54]:
# Define output labels as quantum state vectors
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [55]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs = [x, y]
        x (array[float]): 1-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]


In [56]:
dev_conv = qml.device("default.qubit", wires=9)


@qml.qnode(dev_conv)
def q_conv(conv_params, inputs):
    """A variational quantum circuit representing the Universal classifier + Conv.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    # layer iteration
    for l in range(len(conv_params[0])):
        # RY layer
        # height iteration
        for i in range(3):
            # width iteration
            for j in range(3):
                qml.RY((conv_params[0][l][3*i+j] * inputs[i, j] + conv_params[1][l][3*i+j]), wires=(3*i+j))

        # entangling layer
        for i in range(9):
            if i != (9-1):
                qml.CNOT(wires=[i, i+1])
                
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.PauliZ(4) @ qml.PauliZ(5) @ qml.PauliZ(6) @ qml.PauliZ(7) @ qml.PauliZ(8))

In [57]:
a = np.zeros((2, 1, 9))
q_conv(a, X_train[0, 0:3, 0:3])

tensor(1., requires_grad=True)

In [58]:
a = np.zeros((2, 1, 9))
q_fc(a, X_train[0, 0, 0:9])

tensor([1., 0.], requires_grad=True)

In [59]:
from keras import backend as K


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 2), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [60]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 1
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)

model = tf.keras.Model(inputs=X, outputs=alpha_layer_0)

In [61]:
model(X_train[0:32, :, :])

<tf.Tensor: shape=(32, 2), dtype=float32, numpy=
array([[ 0.02659302, -0.00011611],
       [ 0.02659543, -0.00011406],
       [ 0.02659884, -0.00011116],
       [ 0.02660321, -0.00010745],
       [ 0.02659877, -0.00011122],
       [ 0.02660063, -0.00010964],
       [ 0.02659685, -0.00011285],
       [ 0.02658659, -0.00012158],
       [ 0.02659541, -0.00011408],
       [ 0.0265973 , -0.00011247],
       [ 0.0265956 , -0.00011392],
       [ 0.02659197, -0.000117  ],
       [ 0.02659618, -0.00011342],
       [ 0.02659401, -0.00011527],
       [ 0.02658885, -0.00011966],
       [ 0.02659418, -0.00011512],
       [ 0.02659674, -0.00011295],
       [ 0.02660478, -0.00010611],
       [ 0.02658274, -0.00012485],
       [ 0.0265984 , -0.00011153],
       [ 0.0265921 , -0.00011689],
       [ 0.02659896, -0.00011106],
       [ 0.02659572, -0.00011381],
       [ 0.0266008 , -0.0001095 ],
       [ 0.02660445, -0.0001064 ],
       [ 0.02659565, -0.00011387],
       [ 0.02659516, -0.00011429],
      

In [62]:
import keras.backend as K

# def custom_loss(y_true, y_pred):
#     return K.sum(((y_true.shape[1]-2)*y_true+1)*K.square(y_true-y_pred))/len(y_true)

def custom_loss(y_true, y_pred):
    loss = K.square(y_true-y_pred)
    #class_weights = y_true*(weight_for_1-weight_for_0) + weight_for_0
    #loss = loss * class_weights
    
    return K.sum(loss)/len(y_true)

In [63]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [21]:
cp_val_acc = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/2_QConv2ent_QFC_valacc.hdf5",
                monitor='val_accuracy', verbose=1, save_weights_only=True, save_best_only=True, mode='max')

cp_val_loss = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/2_QConv2ent_QFC_valloss.hdf5",
                monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [22]:
H = model.fit(X_train, to_categorical(Y_train), epochs=10, batch_size=32, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[cp_val_acc, cp_val_loss])

Epoch 1/10
13/13 [==============================] - 6571s 516s/step - loss: 0.4083 - accuracy: 0.4980 - val_loss: 0.2559 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to ./Model/2_QConv2ent_QFC_valacc.hdf5

Epoch 00001: val_loss improved from inf to 0.25595, saving model to ./Model/2_QConv2ent_QFC_valloss.hdf5
Epoch 2/10
13/13 [==============================] - 6628s 522s/step - loss: 0.2677 - accuracy: 0.4488 - val_loss: 0.2473 - val_accuracy: 0.5020

Epoch 00002: val_accuracy improved from 0.50000 to 0.50200, saving model to ./Model/2_QConv2ent_QFC_valacc.hdf5

Epoch 00002: val_loss improved from 0.25595 to 0.24732, saving model to ./Model/2_QConv2ent_QFC_valloss.hdf5
Epoch 3/10
13/13 [==============================] - 6546s 515s/step - loss: 0.2054 - accuracy: 0.6613 - val_loss: 0.1075 - val_accuracy: 0.8790

Epoch 00003: val_accuracy improved from 0.50200 to 0.87900, saving model to ./Model/2_QConv2ent_QFC_valacc.hdf5

Epoch 00003: va

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 27, 27)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 3, 3)         0           Input_Layer[0][0]                
______________________________________________________________________________________________

In [24]:
# fix ent
H.history

{'loss': [0.34813618659973145,
  0.2664911150932312,
  0.16182418167591095,
  0.0643858015537262,
  0.03387075290083885,
  0.023905329406261444,
  0.01728159189224243,
  0.014526670798659325,
  0.015740571543574333,
  0.014256746508181095],
 'accuracy': [0.48500001430511475,
  0.4749999940395355,
  0.8025000095367432,
  0.9325000047683716,
  0.9750000238418579,
  0.9825000166893005,
  0.9775000214576721,
  0.9850000143051147,
  0.9825000166893005,
  0.9825000166893005],
 'val_loss': [0.25594961643218994,
  0.24731574952602386,
  0.1074986532330513,
  0.03673959895968437,
  0.01930546946823597,
  0.01758934184908867,
  0.013311878778040409,
  0.01229886058717966,
  0.015210121870040894,
  0.016688041388988495],
 'val_accuracy': [0.5,
  0.5019999742507935,
  0.8790000081062317,
  0.9710000157356262,
  0.9850000143051147,
  0.9850000143051147,
  0.9850000143051147,
  0.9869999885559082,
  0.9819999933242798,
  0.984000027179718]}

In [25]:
# fix ent
model.weights

[<tf.Variable 'model/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]], dtype=float32)>,
 <tf.Variable 'model/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.531285  , -0.53907233,  1.2279214 , -0.04873934,
          -1.9625113 , -0.31179625,  1.4537607 , -0.3272767 ,
           1.5368826 ]],
 
        [[ 0.8165296 ,  0.3640772 ,  1.7864715 ,  0.5245018 ,
          -1.6975542 , -0.21682522,  2.097541  ,  0.49968052,
           2.2059586 ]]], dtype=float32)>,
 <tf.Variable 'model/keras_layer/params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 0.3726357 , -1.0686979 ,  0.24734715,  2.5706375 ,
           3.4816115 , -0.31677

# Exploring the results

In [26]:
X_train = np.concatenate((x_train_list[0][:20, :], x_train_list[1][:20, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[20:] += 1

X_train.shape, Y_train.shape

((40, 729), (40,))

In [27]:
X_test = np.concatenate((x_test_list[0][:20, :], x_test_list[1][:20, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[20:] += 1

X_test.shape, Y_test.shape

((40, 729), (40,))

In [28]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((40, 27, 27), (40, 27, 27))

## First Layer

In [64]:
qconv_1_weights = np.array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]])

qconv_1_weights.shape

(2, 1, 9)

In [30]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)

qconv1_model = tf.keras.Model(inputs=X, outputs=reshape_layer_1)


In [31]:
qconv1_model(X_train[0:1])
qconv1_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])

In [32]:
qconv1_model.weights

[<tf.Variable 'model_1/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]], dtype=float32)>]

In [33]:
preprocessed_img_train = qconv1_model(X_train)

preprocessed_img_test = qconv1_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2ent_QFC-Filter1_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2ent_QFC-Filter1_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 169) (40, 169)


## Second Layer

In [65]:
qconv_2_weights = np.array([[[ 2.531285  , -0.53907233,  1.2279214 , -0.04873934,
          -1.9625113 , -0.31179625,  1.4537607 , -0.3272767 ,
           1.5368826 ]],
 
        [[ 0.8165296 ,  0.3640772 ,  1.7864715 ,  0.5245018 ,
          -1.6975542 , -0.21682522,  2.097541  ,  0.49968052,
           2.2059586 ]]])

qconv_2_weights.shape

(2, 1, 9)

In [35]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)

qconv2_model = tf.keras.Model(inputs=X, outputs=reshape_layer_2)


In [36]:
qconv2_model(X_train[0:1])
qconv2_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
qconv2_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [37]:
qconv2_model.weights

[<tf.Variable 'model_2/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]], dtype=float32)>,
 <tf.Variable 'model_2/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.531285  , -0.53907233,  1.2279214 , -0.04873934,
          -1.9625113 , -0.31179625,  1.4537607 , -0.3272767 ,
           1.5368826 ]],
 
        [[ 0.8165296 ,  0.3640772 ,  1.7864715 ,  0.5245018 ,
          -1.6975542 , -0.21682522,  2.097541  ,  0.49968052,
           2.2059586 ]]], dtype=float32)>]

In [38]:
preprocessed_img_train = qconv2_model(X_train)

preprocessed_img_test = qconv2_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2ent_QFC-Filter2_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2ent_QFC-Filter2_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 36) (40, 36)


## Quantum States

In [66]:
X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

In [68]:
q_fc_weights = np.array([[[ 0.3726357 , -1.0686979 ,  0.24734715,  2.5706375 ,
           3.4816115 , -0.3167716 ,  1.2021828 ,  0.19082105,
           0.0209959 ]],
 
        [[-0.213224  ,  1.235254  ,  2.298314  ,  2.2815244 ,
          -1.1340573 ,  0.8286312 ,  0.6926544 , -1.0297745 ,
           0.14662777]]])

q_fc_weights.shape

(2, 1, 9)

In [40]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [41]:
np.argmax(pred_train, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], requires_grad=True)

In [42]:
np.argmax(pred_test, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], requires_grad=True)

In [69]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)

maxpool_model = tf.keras.Model(inputs=X, outputs=reshape_layer_3)


In [70]:
maxpool_model(X_train[0:1])
maxpool_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
maxpool_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [71]:
maxpool_train = maxpool_model(X_train)
maxpool_test = maxpool_model(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([400, 9]), TensorShape([1000, 9]))

In [72]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [73]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [74]:
q_fc_state(q_fc_weights, maxpool_train[0])

tensor(0.95763762, requires_grad=True)

In [75]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(q_fc_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
for i in range(len(test_state)):
    q_fc_state(q_fc_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [77]:
train_state.shape, test_state.shape

((400, 2), (1000, 2))

In [78]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [79]:
np.savetxt('./2_QConv2ent_QFC-State_Train_all_samples.txt', train_state)
np.savetxt('./2_QConv2ent_QFC-State_Test_all_samples.txt', test_state)

## Random Starting State

In [80]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 1
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)

model_random = tf.keras.Model(inputs=X, outputs=alpha_layer_0)
model_maxpool_random = tf.keras.Model(inputs=X, outputs=reshape_layer_3)

In [81]:
model_random(X_train[0:1])
model_random.weights

[<tf.Variable 'model_6/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.09383321, -0.15872839,  0.40481728, -0.4808251 ,
           0.0080896 , -0.213866  , -0.07416925, -0.11021015,
          -0.36422676]],
 
        [[ 0.46857107, -0.4836045 , -0.5114353 ,  0.00082922,
          -0.4332028 ,  0.4042952 , -0.48786062, -0.03484303,
           0.35927957]]], dtype=float32)>,
 <tf.Variable 'model_6/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 0.42102844, -0.48002094, -0.33818322, -0.13231662,
           0.34059566,  0.06579536, -0.42546484,  0.44662088,
          -0.11891249]],
 
        [[ 0.35543942,  0.18025482,  0.27855408,  0.22550434,
          -0.36387637,  0.16426843, -0.21730787,  0.16378671,
          -0.45715028]]], dtype=float32)>,
 <tf.Variable 'model_6/keras_layer_3/params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.52412415,  0.31337726, -0.0668911 , -0.43626598,
          -0.10543117, 

In [82]:
random_weights = np.array([[[-0.52412415,  0.31337726, -0.0668911 , -0.43626598,
          -0.10543117, -0.11661649, -0.22867104,  0.26161313,
           0.03067034]],
 
        [[-0.3362496 ,  0.52573454, -0.12595582,  0.4650234 ,
           0.10775095,  0.2522235 ,  0.06523472,  0.32300556,
           0.42630672]]])

In [83]:
maxpool_train = model_maxpool_random(X_train)
maxpool_test = model_maxpool_random(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([400, 9]), TensorShape([1000, 9]))

In [84]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [85]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [86]:
q_fc_state(random_weights, maxpool_train[21])

tensor(0.65946282, requires_grad=True)

In [89]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(random_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
for i in range(len(test_state)):
    
    q_fc_state(random_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [91]:
train_state.shape, test_state.shape

((400, 2), (1000, 2))

In [92]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [93]:
np.savetxt('./2_QConv2ent_QFC-RandomState_Train_all_samples.txt', train_state)
np.savetxt('./2_QConv2ent_QFC-RandomState_Test_all_samples.txt', test_state)

# Finish

In [76]:
first_10_epoch = H.history
first_10_epoch

{'loss': [0.7249730825424194,
  0.485493928194046,
  0.2910504639148712,
  0.20750370621681213,
  0.1200886219739914,
  0.09333930909633636,
  0.09352166950702667,
  0.09991505742073059,
  0.0711403489112854,
  0.0675935298204422],
 'accuracy': [0.5249999761581421,
  0.6225000023841858,
  0.7825000286102295,
  0.8650000095367432,
  0.9449999928474426,
  0.9599999785423279,
  0.9624999761581421,
  0.9524999856948853,
  0.9750000238418579,
  0.9775000214576721],
 'val_loss': [0.6175757646560669,
  0.40504130721092224,
  0.3411727547645569,
  0.19497469067573547,
  0.11362368613481522,
  0.07933302223682404,
  0.1450268030166626,
  0.0960061326622963,
  0.09231595695018768,
  0.053687386214733124],
 'val_accuracy': [0.5,
  0.7300000190734863,
  0.7400000095367432,
  0.8700000047683716,
  0.9449999928474426,
  0.9800000190734863,
  0.925000011920929,
  0.9649999737739563,
  0.9549999833106995,
  0.9750000238418579]}

In [77]:
# initial 10 epoch
model.get_weights()

[array([[[ 0.13920915, -0.7933977 ,  0.4931736 , -0.34984022,
           0.41861725, -0.1258272 , -0.3323837 ,  1.4861933 ,
           0.14679849]],
 
        [[-0.48547882, -0.6672664 , -0.30831295,  0.31500554,
           0.21519628, -0.09552038, -0.33133912,  0.8978053 ,
          -0.38653338]]], dtype=float32),
 array([[[-0.04071528, -0.5614164 , -0.5472108 ,  0.32481748,
          -0.1586527 ,  0.06802678, -0.04312551, -1.0716503 ,
          -0.09168857]],
 
        [[ 0.2695058 , -1.5634912 ,  0.20198631, -0.20597708,
           0.06802317,  0.2522897 , -0.01191515,  0.5282674 ,
          -0.3303873 ]]], dtype=float32),
 array([[[-0.1229878 ,  0.03982117, -0.29502654, -1.2658846 ,
          -0.3401566 , -0.93464977,  1.1978221 , -0.24731389,
          -0.51143396]],
 
        [[-0.05537069, -1.3277841 , -0.7294856 , -0.740143  ,
          -0.83612764,  0.6652604 ,  0.0570471 , -0.4578703 ,
          -0.36179888]]], dtype=float32),
 array([[1.084761 , 1.0133266]], dtype=float32)]